# Using trained anemoi models

The central artifact resulting from the training of an anemoi model is its inference checkpoint*. The checkpoint not only contains the model itself (pickled python object and weights), but also the graph on which the model operates and a rich metadata information that tells us about how the model was trained, what data was used, and more.

In this notebook, we will have a look at the content of a checkpoint and then generate some simple configuration files to launch a model inference run.


*_Contrary to a training checkpoint, an inference checkpoint does not contain training-related information, such as the optimizer's state._


## anemoi-inference

A simple interface for running anemoi models is provided by the anemoi-inference package. As with the other components of anemoi, it is used from the command line. Let's have a look:


**NOTE**: anemoi-inference is one of the less mature components of anemoi - it is still undergoing frequent large changes and refactorings.


In [7]:
!anemoi-inference --help

usage: anemoi-inference [-h] [--version] [--debug] [--rich]
                        {couple,inspect,metadata,patch,requests,retrieve,run,sanitise,validate}
                        ...

positional arguments:
  {couple,inspect,metadata,patch,requests,retrieve,run,sanitise,validate}
                        commands:
    couple              Couple tasks based on a configuration file.
    inspect             Inspect the contents of a checkpoint file.
    metadata            Edit, remove, dump or load metadata from a checkpoint
                        file.
    patch               Patch a checkpoint file.
    requests            MARS request utility.
    retrieve            Used by prepml.
    run                 Run inference from a config yaml file.
    sanitise            Sanitise a checkpoint file.
    validate            Validate the virtual environment against a checkpoint
                        file.

options:
  -h, --help            show this help message and exit
  --version, -V   

As mentioned earlier, we want to have a look at the model's checkpoint. For this, the `inspect` command is provided:

In [8]:
!anemoi-inference inspect --help

usage: anemoi-inference inspect [-h]
                                (--variables | --requirements | --datasets | --dump)
                                [--json] [--validate]
                                path

Inspect the contents of a checkpoint file.

positional arguments:
  path            Path to the checkpoint.

options:
  -h, --help      show this help message and exit
  --variables     List the training variables and their categories.
  --requirements  Print a Python's requirements.txt based on the versions of
                  the packages used during training.
  --datasets      Print the arguments passed to anemoi-dataset during
                  training.
  --dump          Dump information from the checkpoint.
  --json          Output in JSON format (with dump option)
  --validate      Validate the environment.


With this command, we can print information about the checkpoint in the terminal. Let's look at some examples.

In [1]:
CHECKPOINT_PATH="output/training/checkpoint/dfa1cd5d30aa4a1db2aa7404f8a33e72/inference-last.ckpt"

In [6]:
!anemoi-inference inspect --variables ../{CHECKPOINT_PATH} 

/scratch/mch/fzanetta/projects/anemoi-demo-keisler2022/.venv/lib/python3.12/site-packages/anemoi/inference/legacy.py:41: UserWarning: Using legacy _legacy_variables_metadata, please try to patch your weights.
  warnings.warn(f"Using legacy {func.__name__}, please try to patch your weights.")
   10u            => prognostic
   10v            => prognostic
   2d             => prognostic
   2t             => prognostic
   cos_julian_day => computed, forcing
   cos_latitude   => computed, constant, forcing
   cos_local_time => computed, forcing
   cos_longitude  => computed, constant, forcing
   cp             => accumulation, diagnostic
   insolation     => computed, forcing
   lsm            => constant, forcing
   msl            => prognostic
   q_100          => prognostic
   q_1000         => prognostic
   q_150          => prognostic
   q_200          => prognostic
   q_250          => prognostic
   q_300          => prognostic
   q_400          => prognostic
   q_50           => pr